# Cluster Optimization for Embeddings Data

This notebook implements:
1. Data cleaning to remove noise items
2. Methods to find the optimal number of clusters
3. Final clustering with the best parameters

## 1. Data Loading and Preparation

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette('viridis')

In [ ]:
# Load the data
df = pd.read_csv('data_w_embeddings.csv')
print(f"Original data shape: {df.shape}")
df.head()

In [ ]:
# Filter out rows with null embeddings
df_filtered = df[df['embedding'].notnull()]
print(f"Filtered data shape: {df_filtered.shape}")
print(f"Removed {df.shape[0] - df_filtered.shape[0]} rows with null embeddings")

# Convert embeddings from strings to numpy arrays
df_filtered["embedding"] = df_filtered.embedding.apply(literal_eval).apply(np.array)

# Create embedding matrix
matrix = np.vstack(df_filtered.embedding.values)
print(f"Embedding matrix shape: {matrix.shape}")

## 2. Data Cleaning - Removing Noise Items

### 2.1 Method 1: Distance-based Outlier Detection

We'll calculate the average distance of each point to its k nearest neighbors and remove points with unusually large distances.

In [ ]:
from sklearn.neighbors import NearestNeighbors

def detect_distance_outliers(matrix, n_neighbors=50, threshold=1.0):
    """
    Detect outliers based on the average distance to k nearest neighbors.

    Parameters:
    - matrix: Embedding matrix
    - n_neighbors: Number of neighbors to consider
    - threshold: Threshold for z-score to identify outliers

    Returns:
    - is_inlier: Boolean array indicating which points are inliers
    """
    # Find k nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(matrix)
    distances, _ = nbrs.kneighbors(matrix)

    # Calculate average distance to k nearest neighbors
    avg_distances = distances[:, 1:].mean(axis=1)  # Exclude the point itself (distance=0)

    # Calculate z-scores
    z_scores = (avg_distances - avg_distances.mean()) / avg_distances.std()

    # Identify outliers
    is_inlier = z_scores < threshold

    return is_inlier

# Apply distance-based outlier detection
is_inlier_distance = detect_distance_outliers(matrix)
print(f"Identified {(~is_inlier_distance).sum()} outliers out of {matrix.shape[0]} points using distance-based method")

### 2.2 Method 2: Local Outlier Factor (LOF)

LOF measures the local deviation of density of a given sample with respect to its neighbors.

In [ ]:
# Apply Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=50, contamination=0.1)
is_inlier_lof = lof.fit_predict(matrix) == 1  # 1 for inliers, -1 for outliers
print(f"Identified {(~is_inlier_lof).sum()} outliers out of {matrix.shape[0]} points using LOF")

### 2.3 Method 3: Isolation Forest

Isolation Forest isolates observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

In [ ]:
# Apply Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
is_inlier_isoforest = iso_forest.fit_predict(matrix) == 1  # 1 for inliers, -1 for outliers
print(f"Identified {(~is_inlier_isoforest).sum()} outliers out of {matrix.shape[0]} points using Isolation Forest")

### 2.4 Combine Methods and Visualize Results

In [ ]:
# Combine results from all methods (conservative approach - a point is an outlier if identified by at least 2 methods)
outlier_votes = (~is_inlier_distance).astype(int) + (~is_inlier_lof).astype(int) + (~is_inlier_isoforest).astype(int)
is_inlier_combined = outlier_votes < 2

print(f"Identified {(~is_inlier_combined).sum()} outliers out of {matrix.shape[0]} points using combined methods")

# Create a clean dataset
df_clean = df_filtered[is_inlier_combined].copy()
print(f"Clean data shape: {df_clean.shape}")

# Create clean embedding matrix
matrix_clean = np.vstack(df_clean.embedding.values)
print(f"Clean embedding matrix shape: {matrix_clean.shape}")

In [ ]:
# Visualize original vs. clean data using t-SNE
def plot_tsne_comparison(matrix_original, matrix_clean, perplexity=30):
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))

    # Original data
    tsne_original = TSNE(n_components=2, perplexity=perplexity, random_state=42, init="random", learning_rate=200)
    vis_original = tsne_original.fit_transform(matrix_original)
    axes[0].scatter(vis_original[:, 0], vis_original[:, 1], alpha=0.5)
    axes[0].set_title(f"Original Data (n={matrix_original.shape[0]})")

    # Clean data
    tsne_clean = TSNE(n_components=2, perplexity=perplexity, random_state=42, init="random", learning_rate=200)
    vis_clean = tsne_clean.fit_transform(matrix_clean)
    axes[1].scatter(vis_clean[:, 0], vis_clean[:, 1], alpha=0.5)
    axes[1].set_title(f"Clean Data (n={matrix_clean.shape[0]})")

    plt.tight_layout()
    return fig

# Plot comparison
plot_tsne_comparison(matrix, matrix_clean)

## 3. Finding Optimal Number of Clusters

### 3.1 Elbow Method

The elbow method looks at the percentage of variance explained as a function of the number of clusters. The optimal number of clusters is the one where adding another cluster doesn't give much better modeling of the data.

In [ ]:
def plot_elbow_method(matrix, max_clusters=80):
    inertia = []
    K_range = range(1, max_clusters + 1)

    for k in K_range:
        kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, n_init=10)
        kmeans.fit(matrix)
        inertia.append(kmeans.inertia_)

    # Plot elbow curve
    plt.figure(figsize=(12, 6))
    plt.plot(K_range, inertia, 'bo-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia (Sum of Squared Distances)')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)

    # Calculate the rate of decrease
    decrease_rate = np.array([inertia[i-1] - inertia[i] for i in range(1, len(inertia))])
    normalized_decrease = decrease_rate / inertia[0]

    # Find the elbow point (where the rate of decrease slows down significantly)
    elbow_point = np.argmax(np.diff(normalized_decrease)) + 1

    plt.axvline(x=elbow_point + 1, color='r', linestyle='--', label=f'Elbow Point: k={elbow_point + 1}')
    plt.legend()

    return elbow_point + 1

# Apply elbow method
optimal_k_elbow = plot_elbow_method(matrix_clean)
print(f"Optimal number of clusters according to elbow method: {optimal_k_elbow}")

### 3.2 Silhouette Analysis

The silhouette value measures how similar an object is to its own cluster compared to other clusters. The silhouette ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

In [ ]:
def plot_silhouette_analysis(matrix, max_clusters=80):
    K_range = range(2, max_clusters + 1)  # Silhouette requires at least 2 clusters
    silhouette_scores = []

    for k in K_range:
        kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, n_init=10)
        labels = kmeans.fit_predict(matrix)
        silhouette_avg = silhouette_score(matrix, labels)
        silhouette_scores.append(silhouette_avg)

    # Plot silhouette scores
    plt.figure(figsize=(12, 6))
    plt.plot(K_range, silhouette_scores, 'bo-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Analysis for Optimal k')
    plt.grid(True)

    # Find the optimal number of clusters
    optimal_k = K_range[np.argmax(silhouette_scores)]
    plt.axvline(x=optimal_k, color='r', linestyle='--', label=f'Optimal k: {optimal_k}')
    plt.legend()

    return optimal_k

# Apply silhouette analysis
optimal_k_silhouette = plot_silhouette_analysis(matrix_clean)
print(f"Optimal number of clusters according to silhouette analysis: {optimal_k_silhouette}")

### 3.3 Calinski-Harabasz Index

The Calinski-Harabasz Index (also known as the Variance Ratio Criterion) is the ratio of the sum of between-cluster dispersion and within-cluster dispersion. Higher values indicate better clustering.

In [ ]:
def plot_calinski_harabasz(matrix, max_clusters=80):
    K_range = range(2, max_clusters + 1)  # CH index requires at least 2 clusters
    ch_scores = []

    for k in K_range:
        kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, n_init=10)
        labels = kmeans.fit_predict(matrix)
        ch_score = calinski_harabasz_score(matrix, labels)
        ch_scores.append(ch_score)

    # Plot CH scores
    plt.figure(figsize=(12, 6))
    plt.plot(K_range, ch_scores, 'bo-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Calinski-Harabasz Index')
    plt.title('Calinski-Harabasz Index for Optimal k')
    plt.grid(True)

    # Find the optimal number of clusters
    optimal_k = K_range[np.argmax(ch_scores)]
    plt.axvline(x=optimal_k, color='r', linestyle='--', label=f'Optimal k: {optimal_k}')
    plt.legend()

    return optimal_k

# Apply Calinski-Harabasz index
optimal_k_ch = plot_calinski_harabasz(matrix_clean)
print(f"Optimal number of clusters according to Calinski-Harabasz index: {optimal_k_ch}")

### 3.4 Davies-Bouldin Index

The Davies-Bouldin Index is the average similarity measure of each cluster with its most similar cluster. Lower values indicate better clustering.

In [ ]:
def plot_davies_bouldin(matrix, max_clusters=80):
    K_range = range(2, max_clusters + 1)  # DB index requires at least 2 clusters
    db_scores = []

    for k in K_range:
        kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, n_init=10)
        labels = kmeans.fit_predict(matrix)
        db_score = davies_bouldin_score(matrix, labels)
        db_scores.append(db_score)

    # Plot DB scores
    plt.figure(figsize=(12, 6))
    plt.plot(K_range, db_scores, 'bo-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Davies-Bouldin Index')
    plt.title('Davies-Bouldin Index for Optimal k')
    plt.grid(True)

    # Find the optimal number of clusters (minimum DB score)
    optimal_k = K_range[np.argmin(db_scores)]
    plt.axvline(x=optimal_k, color='r', linestyle='--', label=f'Optimal k: {optimal_k}')
    plt.legend()

    return optimal_k

# Apply Davies-Bouldin index
optimal_k_db = plot_davies_bouldin(matrix_clean)
print(f"Optimal number of clusters according to Davies-Bouldin index: {optimal_k_db}")

### 3.5 Compare Results from Different Methods

In [ ]:
# Summarize results
methods = ['Elbow Method', 'Silhouette Analysis', 'Calinski-Harabasz Index', 'Davies-Bouldin Index']
optimal_k_values = [optimal_k_elbow,
                    optimal_k_silhouette, optimal_k_ch, optimal_k_db]

summary_df = pd.DataFrame({
    'Method': methods,
    'Optimal Number of Clusters': optimal_k_values
})

print("Summary of optimal cluster numbers:")
summary_df

In [ ]:
# Determine the final optimal number of clusters (e.g., by taking the mode or median)
from scipy import stats

final_optimal_k = int(stats.mode(optimal_k_values)[0])  # Mode
print(f"Final optimal number of clusters (mode): {final_optimal_k}")

median_optimal_k = int(np.median(optimal_k_values))  # Median
print(f"Final optimal number of clusters (median): {median_optimal_k}")

## 4. Final Clustering with Optimal Parameters

In [ ]:
# Apply K-means with the optimal number of clusters
optimal_k = median_optimal_k  # Use the mode as the final optimal k
kmeans = KMeans(n_clusters=optimal_k, init="k-means++", random_state=42, n_init=10)
labels = kmeans.fit_predict(matrix_clean)

# Add cluster labels to the dataframe
df_clean["Cluster"] = labels

# Display cluster sizes
cluster_sizes = df_clean.groupby("Cluster").size().sort_values(ascending=False)
print("Cluster sizes:")
print(cluster_sizes)

In [ ]:
# Visualize the final clusters using t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42, init="random", learning_rate=200)
vis_dims = tsne.fit_transform(matrix_clean)

# Create a scatter plot
plt.figure(figsize=(12, 10))

# Use a colormap with distinct colors
colors = plt.cm.tab20(np.linspace(0, 1, optimal_k))

for cluster_id in range(optimal_k):
    cluster_points = vis_dims[labels == cluster_id]
    plt.scatter(
        cluster_points[:, 0],
        cluster_points[:, 1],
        color=colors[cluster_id],
        alpha=0.7,
        # label=f'Cluster {cluster_id}'
    )

    # Add cluster centroid
    centroid = np.mean(cluster_points, axis=0)
    plt.scatter(
        centroid[0],
        centroid[1],
        marker='X',
        s=200,
        color=colors[cluster_id],
        edgecolor='black'
    )

plt.title(f"t-SNE Visualization of {optimal_k} Clusters")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### 4.1 Analyze Cluster Characteristics

In [ ]:
# Display sample items from each cluster
for cluster_id in range(optimal_k):
    cluster_samples = df_clean[df_clean['Cluster'] == cluster_id].head(5)
    print(f"\nCluster {cluster_id} samples:")
    if 'text' in cluster_samples.columns:
        for _, row in cluster_samples.iterrows():
            print(f"ID: {row['_id']}")
            print(f"Text: {row['text'][:200]}..." if len(str(row['text'])) > 200 else f"Text: {row['text']}")
            print("---")
    else:
        display(cluster_samples)

### 4.2 Save Results

In [ ]:
# Save the cleaned and clustered data
output_columns = ['_id', 'Cluster']
if 'text' in df_clean.columns:
    output_columns.insert(2, 'text')

output_filename = f'data_cleaned_clustered_k{optimal_k}.csv'
df_clean.to_csv(output_filename, columns=output_columns, index=False)
print(f"Saved cleaned and clustered data to {output_filename}")